In [1]:
'''
IMPORTANT INSTRUCTIONS!!!

1. Before Proceeding, add dataset : www.kaggle.com/srikaranand/road-segmentation-dataset
   (click 'Add Data' button and then 'click search from URL')

2. To run the program,  run cells #11 , #12 and #13  (Cell numbers at top of each cell)

'''



#1
%matplotlib inline
import itertools
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import losses
from keras.datasets import mnist
from keras.models import Sequential, model_from_json

from keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, LeakyReLU
from keras import backend as K
from keras.optimizers import Adam

#from keras.utils.training_utils import multi_gpu_model

#from helpers import *

Using TensorFlow backend.


In [ ]:
#2
print(keras.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#3
imgs, gt_imgs = training_dataset(limit=100)
print('Image size =', imgs[0].shape)

In [ ]:
#4
patch_size = 16 # each patch is 16*16 pixels

In [ ]:
#5
train_data = [
    (patch, gt_patch)
    for i in range(len(imgs))
    for patch, gt_patch initertools.islice(gen_random_patches(reflect_border(imgs[i], patch_size, 2),
                                                              reflect_border(gt_imgs[i], patch_size, 2), i), 25*25*8)]


X = np.asarray([img for img, _ in train_data])
y = np.asarray([value_to_class(gt_patch) for _, gt_patch in train_data]).reshape((-1, 1))

print(X.shape)
print(y.shape)

In [ ]:
#6
# Model parameters

batch_size = 64 * 8
num_filters_1 = 16
num_filters_2 = 32
num_filters_3 = 64
num_filters_4 = 128

epochs = 50

In [ ]:
#7
def build_model(gpus=0):
    model = Sequential()
    model.add(Conv2D(num_filters_1, kernel_size=(4, 4), input_shape=(80, 80, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_2, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_3, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_4, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    simple_model = model
    if gpus > 1:
        model = multi_gpu_model(simple_model, gpus=gpus)

    model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
    
    return (model, simple_model)

In [ ]:
#8
model, simple_model = build_model(0)
model.summary()

In [ ]:
#9
model.fit(X, y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True)

In [ ]:
#10
# save the model and the weights to files
model_json = simple_model.to_json()
with open('savedModels/model_18', 'w') as f:
    f.write(model_json)

simple_model.save_weights('savedModels/weights_18')

In [3]:
#11
#helpers.py
!pip install imutils
import os,sys
import re
import cv2
#import imutils
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import random

patch_size = 16



def load_image(infilename):
    data = mpimg.imread(infilename)
    return data


def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg


def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[i:i+w, j:j+h]
            else:
                im_patch = im[i:i+w, j:j+h, :]
            list_patches.append(im_patch)
    return list_patches


def reflect_border(im, patch_size, count):
    bordersize = patch_size*count
    border = cv2.copyMakeBorder(im,
                              top=bordersize,
                              bottom=bordersize, 
                              left=bordersize,
                              right=bordersize,
                              borderType=cv2.BORDER_REFLECT)
    return border

def rotate_image(img, rot_degrees=[0, 90, 180, 270]):
    return [
        imutils.rotate_bound(img, rot)
        for rot in rot_degrees
    ]


def flip_image(img):
    return cv2.flip(img, 1)


def image_trans(img):
    return [
        t_img
        for rot_img in rotate_image(img)
        for t_img in [rot_img, flip_image(rot_img)]
    ]


def training_dataset(limit=100, root_dir='input/road-segmentation-dataset/data/training/'):
    image_dir = root_dir + "images/"
    files = os.listdir(image_dir)
    n = min(limit, len(files))
    print('Loading', n, 'training images')
    imgs = [load_image(image_dir + files[i]) for i in range(n)]
    gt_dir = root_dir + "groundtruth/"
    print('Loading', n, 'groundtruth images')
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
    return (imgs, gt_imgs)

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[i:i+w, j:j+h] = labels[idx]
            idx = idx + 1
    return im

def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*255

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

def print_prediction(img, gt_img, model):
    img_patches = [
        img_patch
        for img_patch in image_to_patches(reflect_border(img, patch_size, 2), get_image_crop)
    ]

    gt_labeled_patches = [
        value_to_class(gt_patch)
        for gt_patch in image_to_patches(reflect_border(gt_img, patch_size, 2), get_groundtruth_crop)
    ]

    predicted_patches = model.predict(np.asarray(img_patches))
    
    w = img.shape[0]
    h = img.shape[1]
    predicted_im = label_to_img(w, h, patch_size, patch_size, predicted_patches)
    
    
    print("Ground truth")
    cimg = concatenate_images(img, gt_img)
    fig1 = plt.figure(figsize=(10, 10))
    plt.imshow(cimg, cmap='Greys_r')
    plt.show()
    
    print("Ground truth patches")
    gt_im_patches = label_to_img(w, h, patch_size, patch_size, gt_labeled_patches)

    cimg = concatenate_images(img, gt_im_patches)
    fig1 = plt.figure(figsize=(10, 10)) 
    plt.imshow(cimg, cmap='Greys_r')
    plt.show()
    
    print("Predicted patches")

    cimg = concatenate_images(img, predicted_im)
    fig1 = plt.figure(figsize=(10, 10))  
    plt.imshow(cimg, cmap='Greys_r')
    plt.show()
    
    print("Prediction overlay")

    new_img = make_img_overlay(img, predicted_im)

    plt.imshow(new_img)

def image_to_patches(img, func):
    p = img.shape
    assert p[0] == p[1]
    n = p[0]
    patches = [
        func(img, i, j)
        for i in range(2*patch_size, n - 3*patch_size+1, patch_size)
        for j in range(2*patch_size, n - 3*patch_size+1, patch_size)
    ]
    return patches

def get_image_crop(img, i, j):
    return img[i-2*patch_size:i+3*patch_size, j-2*patch_size:j+3*patch_size, :]

def get_groundtruth_crop(gt_img, i, j):
    return gt_img[i:i+patch_size, j:j+patch_size]

def value_to_class(v, foreground_threshold = 0.25):
    df = np.mean(v)
    return (df > foreground_threshold) * 1


def gen_random_patches(img, gt, seed=42):
    random.seed(seed)
    n = img.shape[0]
    
    angle = random.choice([0, 90, 180, 270])
    
    rot_img = rotate_image(img, [angle])[0]
    rot_gt = rotate_image(gt, [angle])[0]
    
    if random.randrange(2) == 1:
        rot_img = flip_image(rot_img)
        rot_gt = flip_image(rot_gt)
                           
    while True:
        x = random.randrange(n - 80)
        y = random.randrange(n - 80)
        yield (
            rot_img[x:x+80,y:y+80,:],
            rot_gt[x+2*patch_size:x+3*patch_size,
                   y+2*patch_size:y+3*patch_size]
        )

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[i:i+w, j:j+h] = labels[idx]
            idx = idx + 1
    return im
            
            
def img_to_submission_strings(labels, img_number, w, h, patch_size):
    """outputs the strings that should go into the submission file for a given image"""
    label_img = labels.reshape(w//patch_size,h//patch_size)
    for j in range(0, w, patch_size):
        for i in range(0, h, patch_size):
            label = label_img[i//patch_size][j//patch_size]
            if label >= 0.5:
                sub_lab = 1
            else:
                sub_lab = 0
                
            yield("{:03d}_{}_{},{}".format(img_number, j, i, sub_lab))

            
def image_to_inputs(img, patch_size):
    rows, cols, _ = img.shape
    
    patches = [
        
            img[i-2*patch_size:i+3*patch_size, j-2*patch_size:j+3*patch_size, :]
        
        for i in range(2*patch_size, rows - 3*patch_size+1, patch_size)
        for j in range(2*patch_size, cols - 3*patch_size+1, patch_size)
    ]

    return patches


def disp_img_pred(img, pred, patch_size):
    w = img.shape[0]
    h = img.shape[1]
    predicted_im = label_to_img(w, h, patch_size, patch_size, pred)
    new_img = make_img_overlay(img, predicted_im)
    fig1 = plt.figure(figsize=(10,10))
    plt.imshow(new_img)
    plt.show()


def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*255
    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [4]:
#12
#run.py

import itertools
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras import losses
from keras.datasets import mnist
from keras.models import Sequential, model_from_json

from keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, MaxPooling2D, LeakyReLU
from keras import backend as K
from keras.optimizers import Adam

import imutils


#from helpers import *

patch_size = 16 #size of each batch of pixels


imgs, gt_imgs = training_dataset(limit=10)

train_data = [
    (patch, gt_patch)
    for i in range(len(imgs))
    for patch, gt_patch in itertools.islice(gen_random_patches(reflect_border(imgs[i], patch_size, 2),
                                                              reflect_border(gt_imgs[i], patch_size, 2), i), 25*25*8)]


X = np.asarray([img for img, _ in train_data])
y = np.asarray([value_to_class(gt_patch) for _, gt_patch in train_data]).reshape((-1, 1))


batch_size = 64
num_filters_1 = 16
num_filters_2 = 32
num_filters_3 = 64
num_filters_4 = 128

epochs = 50

def build_model(gpus=0):
    model = Sequential()
    model.add(Conv2D(num_filters_1, kernel_size=(4, 4), input_shape=(80, 80, 3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_2, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_3, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(num_filters_4, kernel_size=(4,4)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    simple_model = model
    if gpus > 1:
        model = multi_gpu_model(simple_model, gpus=gpus)

    model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),
              metrics=['accuracy'])
    
    return (model, simple_model)

model, simple_model = build_model(0)
model.summary()



#model.fit(X, y,batch_size=batch_size, epochs=epochs,verbose=1, shuffle=True)


#model_json = simple_model.to_json()
#with open('model_18', 'w') as f:
#    f.write(model_json)

#simple_model.save_weights('weights_18')


fm = open('input/road-segmentation-dataset/model_18') #pre-trained weights
model = model_from_json(fm.read())
fm.close()

model.load_weights('input/road-segmentation-dataset/weights_18', by_name=False)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-3),#'rmsprop',
              metrics=['accuracy'])


foreground_threshold = 0.5 
patch_size = 16

def pred_print(img, img_patches):
    print('Prediction started...')
    pred = [
        np.round((np.median(model.predict(np.asarray(image_trans(patch))))))
        for patch in img_patches
    ]
    pred = np.asarray(pred)
    print('Prediction done...')
    #disp_img_pred(img, pred, patch_size)
    return pred


def submission_with_model(model, submissionfilename):

    root_dir = "input/road-segmentation-dataset/data/test_set_images/"

    
    image_dir = [root_dir + "test_{}/".format(i) for i in range(1, 51)]
    filenames = [fn for imdir in image_dir for fn in os.listdir(imdir)]
    images = [load_image(image_dir[i-1] + filenames[i-1]) for i in range(1, 51)]

    
    im_borders = [reflect_border(im, patch_size, 2) for im in images]
    
    
    imgs_patched = [image_to_inputs(im, patch_size) for im in im_borders]
    
    
    n_images = 1   #number of test images to predict


    predictions = []
    for i in range(1,n_images+1):
        predictions.append(pred_print(images[i-1], np.asarray(imgs_patched[i-1])))
    
    with open(submissionfilename, 'w') as f:
        for nr in range(1, n_images+1):
            f.writelines(
                '{}\n'.format(s)
                for s in img_to_submission_strings(predictions[nr-1],
                                                   nr,
                                                   images[nr-1].shape[1],
                                                   images[nr-1].shape[0],
                                                   patch_size))

submission_with_model(model, 'output.csv')

Using TensorFlow backend.
Loading 10 training images
Loading 10 groundtruth images
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 77, 77, 16)        784       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 77, 77, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 35, 35, 32)        8224      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 35, 35, 32)        0         
_____________________________________

In [7]:
#13
#csv2img.py

# This program converts the csv output to images

import csv
from PIL import Image


n_images = 1

images=[]
for _ in range(n_images):
  img = Image.new('RGB',[593,593])
  images.append(img)
  #print(img1.mode)

with open('output.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        records = row[0].split('_')
        value = int(row[1])*255
        i = int(records[0])-1
        x = int(records[1])
        y = int(records[2])
        img = images[i].load()
        img[x,y] = (value, value,value)

img_num=0
for img1 in images:
  output = img1
  output_ = output.load()
  for i in range(img1.size[0]):
    for j in range(img1.size[1]):
      x = (i//16) * 16
      y = (j//16) * 16
      output_[i,j] = img[x,y]
  img_num=img_num+1
  output.save('output/mask_'+str(img_num)+'.png')
print('Images saved in output directory')

Images saved in output directory


In [8]:
#14
#masking
from PIL import Image

color = (0,0,0)

def mask(pic,mask,outpic):
    img = Image.open(pic)
    mask = Image.open(mask)
    img = img.resize(mask.size)
    img_pix = img.load()
    mask_pix = mask.load()
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            if sum(mask_pix[i,j])/3 > 127:
                img_pix[i,j] = color
    img.save(outpic)

mask('input/road-segmentation-dataset/data/test_set_images/test_1/test_1.png','output/mask_1.png','output/final_output_1.png')